In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld

URM_all, _, _, _, _ = ld.getCOOs()
ICM_all = ld.getICMall()

SEED = 8888

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=SEED)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [33]:
import os

ofp = "../models_temp/Similarity_Hybrid/"


models_to_combine_best = {
                            'IALS' : {"num_factors": 29, "epochs": 50, "confidence_scaling": "log", "alpha": 0.001, "epsilon": 0.001, "reg": 0.01},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'ICM_all': {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'TopPop': {},
                            'PureSVD': {'num_factors': 30},
                            'UserKNN': {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'EASE_R': {'l2_norm': 2700},
}

In [6]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [7]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

In [8]:
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender

In [34]:
recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender2 = SLIM_BPR_Cython(URM_train)
recommender3 = RP3betaRecommender(URM_train)
recommender4 = ItemKNNCBFRecommender(URM_train, ICM_all)
recommender5 = IALSRecommender(URM_train)
#recommender6 = P3alphaRecommender(URM_train)
recommender7 = UserKNNCFRecommender(URM_train)
#recommender8 = PureSVDRecommender(URM_train)
recommender9 = TopPop(URM_train)
recommender10 = EASE_R_Recommender(URM_train)

model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
model_init(recommender2, 'SLIMBPR_8888', models_to_combine_best['SLIMBPR'])
model_init(recommender3, 'RP3beta', models_to_combine_best['RP3beta'])
model_init(recommender4, 'ICM_all', models_to_combine_best['ICM_all'])
model_init(recommender5, 'IALS', models_to_combine_best['IALS'])
#model_init(recommender6, 'P3alpha', models_to_combine_best['P3alpha'])
model_init(recommender7, 'UserKNN', models_to_combine_best['UserKNN'])
#model_init(recommender8, 'PureSVD', models_to_combine_best['PureSVD'])
model_init(recommender9, 'TopPop', models_to_combine_best['TopPop'])
model_init(recommender10, 'EASE_R', models_to_combine_best['EASE_R'])

NameError: name 'EASE_R_Recommender' is not defined

In [26]:
#  IALS + ElasticNet

recommenderHybrid1 = ItemKNNScoresHybridTwoRecommender(URM_train, recommender4, recommender9)
recommenderHybrid1.fit(alpha = 0.65)

recommenderHybrid2 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid1, recommender3)
recommenderHybrid2.fit(alpha = 0.10)

recommenderHybrid3 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid2, recommender2)
recommenderHybrid3.fit(alpha = 0.20)

recommenderHybrid4 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid3, recommender7)
recommenderHybrid4.fit(alpha = 0.25)

recommenderHybrid5 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid4, recommender5)
recommenderHybrid5.fit(alpha = 0.9)

In [14]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [15]:
rank_models(evaluator_validation, [recommender1, recommender2, recommender3, recommender4, recommender5, recommender6, recommender7, recommender8, recommender9])

EvaluatorHoldout: Processed 13648 (100.0%) in 14.35 sec. Users per second: 951
SLIMElasticNetRecommender 0.24901789784886216
EvaluatorHoldout: Processed 13648 (100.0%) in 32.30 sec. Users per second: 422
SLIM_BPR_Recommender 0.22886888479502734


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [29:04<00:00, 10.35it/s]


EvaluatorHoldout: Processed 13648 (100.0%) in 15.62 sec. Users per second: 874
RP3betaRecommender 0.21789663349708868
EvaluatorHoldout: Processed 13648 (100.0%) in 22.29 sec. Users per second: 612
ItemKNNCBFRecommender 0.08534759729898106
EvaluatorHoldout: Processed 13648 (100.0%) in 4.21 sec. Users per second: 3242
IALSRecommender 0.2322583429864711
EvaluatorHoldout: Processed 13648 (100.0%) in 23.96 sec. Users per second: 570
P3alphaRecommender 0.20192505384822976
EvaluatorHoldout: Processed 13648 (100.0%) in 11.34 sec. Users per second: 1204
UserKNNCFRecommender 0.2309831168936895
EvaluatorHoldout: Processed 13648 (100.0%) in 3.78 sec. Users per second: 3613
PureSVDRecommender 0.23126866079569675
EvaluatorHoldout: Processed 13648 (100.0%) in 3.58 sec. Users per second: 3808
TopPopRecommender 0.1689354158525456


In [30]:
import numpy as np

# TWO RECOMMENDERS

FROM = 0.91
TO = 0.99
# 2341 at 0.7
def test_percentage(recommender_1, recommender_2, evaluator, step):
    recommender = ItemKNNScoresHybridTwoRecommender(URM_train, recommender_1, recommender_2)
    results = []
    alp_space = np.linspace(FROM, TO, step, True)
    for alp in alp_space:
        recommender.fit(alpha = alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [31]:
STEP = 6
alp_space, results = test_percentage(recommenderHybrid4, recommender5, evaluator_validation, STEP)

EvaluatorHoldout: Processed 6000 (44.0%) in 5.04 min. Users per second: 20
EvaluatorHoldout: Processed 12000 (88.0%) in 10.09 min. Users per second: 20
EvaluatorHoldout: Processed 13643 (100.0%) in 11.36 min. Users per second: 20
0.91 : 0.24256186671692603
EvaluatorHoldout: Processed 7000 (51.3%) in 5.83 min. Users per second: 20
EvaluatorHoldout: Processed 13643 (100.0%) in 11.22 min. Users per second: 20
0.926 : 0.24235534473751796
EvaluatorHoldout: Processed 6000 (44.0%) in 5.01 min. Users per second: 20
EvaluatorHoldout: Processed 13000 (95.3%) in 10.77 min. Users per second: 20
EvaluatorHoldout: Processed 13643 (100.0%) in 11.22 min. Users per second: 20
0.9420000000000001 : 0.24204989709240998


KeyboardInterrupt: 

In [ ]:
import numpy as np

# THREE RECOMMENDERS

FROM = 0.1
TO = 0.9

def test_percentage(recommender_1, recommender_2, recommender_3, evaluator, step):
    recommender = ItemKNNScoresHybridMultipleRecommender(URM_train, recommender_1, recommender_2, recommender_3)
    results = []
    alp_space = np.linspace(12, 18, step, True)
    for alp in alp_space:
        recommender.fit(alpha = alp, beta = 5, gamma = 1)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
alp_space, results = test_percentage(recommender1, recommender2, recommender3, evaluator_validation, 5)

In [ ]:
import matplotlib.pyplot as plt

_ = plt.figure(figsize=(16, 9))
plt.plot(alp_space,results, label='MAP variability')
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()

plt.savefig('userwise.png')

* # SLIMER + KNNweight: 0.2499 with alpha = 0.95
* # SLIMER + SLIMBPR: 0.2507 with alpha = 0.58
* # (SLIMER + SLIMBPR) + KNN_selected_3: 0.2509 with alpha 0.96

In [ ]:
hyp = {'alpha': 0.9674267406603186, 'beta': 0.017675237513072553, 'gamma': 0.04900644363228254}

recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all)
# recommender1.fit(**models_to_combine_best['SLIMER'])
# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')

recommender2 = SLIM_BPR_Cython(URM_all)
# recommender2.fit(**models_to_combine_best['SLIMBPR'])
# recommender2.save_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')
recommender2.load_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')

recommender3 = RP3betaRecommender(URM_all)
# recommender3.fit(**models_to_combine_best['RP3beta'])
# recommender3.save_model(ofp + 'RP3beta_ALL/', 'RP3beta_ALL')
recommender3.load_model(ofp + 'RP3beta_ALL/', 'RP3beta_ALL')

recommenderHybrid = ItemKNNScoresHybridMultipleRecommender(URM_all, recommender1, recommender2, recommender3)
recommenderHybrid.fit(**hyp)

recommender = recommenderHybrid

import pandas as pd
at = 10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
sep=",",
dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=at)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


	# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
	# submission

from datetime import datetime

now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)



In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
CFrecommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
CFrecommender.load_model('../result_experiments/BEST_SLIM_PURECF/', file_name = CFrecommender.RECOMMENDER_NAME + "_my_own_save.zip")

from Recommenders.FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg
FW_recommender = CFW_D_Similarity_Linalg(URM_train, ICM_stacked, CFrecommender.W_sparse)
FW_recommender.load_model('../result_experiments/BEST_FW/', file_name = FW_recommender.RECOMMENDER_NAME + "_my_own_save.zip")

In [ ]:
import scipy.sparse as sps
ICM_length_all_5km = ld.getICMlength('5km')
ICM_stacked = sps.hstack((ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km))

In [ ]:
import pandas as pd
argsort_features = np.argsort(-FW_recommender.D_best)

selection_quota = 0.1
        
n_to_select = int(selection_quota*len(argsort_features))
selected_features = argsort_features[:n_to_select]
        
ICM_selected = ICM_stacked.todense()[:,selected_features]
pd.DataFrame(ICM_selected).to_csv('ICM_selected_1.csv', index=False, header=True)


In [ ]:
import scipy.sparse as sps
import pandas as pd
URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, _ = ld.getCOOs()
ICM_length_all_5km = ld.getICMlength('5km')
pd.DataFrame(sps.hstack([ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km]).todense()).to_csv('ICM_ALL.csv', index=False, header=True)

In [ ]:
recommender1 = recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])
recommender1.fit(**models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])

# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
# recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')